In [1]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout
from tensorflow.keras.layers import BatchNormalization, Activation, ZeroPadding2D
from tensorflow.keras.layers import LeakyReLU, Conv2DTranspose
from tensorflow.keras.layers import UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

import matplotlib.pyplot as plt

import sys

import numpy as np
from tqdm import tqdm

In [3]:
num_of_samples = 16
z = 100
epochs = 50
batch_size = 256

input_noise = tf.random.normal([16, z])
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)


# discriminator
def discriminator_loss(real_output, fake_output):

    real_loss = cross_entropy(tf.ones_like(real_output), fake_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss

    return total_loss



In [4]:
def generator_loss(fake_output):
  return cross_entropy(tf.ones_like(fake_output), fake_output)

In [27]:
from tensorflow.python import summary

class DCGAN():
  def __init__(self, rows, cols, channels, z=100):
    # this is the input shape
    self.img_rows = rows
    self.img_cols = cols
    self.channels = channels
    self.img_shape = (self.img_rows, self.img_cols, self.channels)
    self.latent_dim = z

    # build and compile the discriminator
    # ref to the funcion down below
    self.discriminator = self.build_discriminator()


    # this is the generator
    # ref to the function down below
    self.generator = self.build_generator()
    self.generator_optimizer = tf.keras.optimizers.Adam(1e-4)
    self.discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)


     # Batch normalization applies a transformation that maintains
     # the mean output close to 0 and the output standard deviation close to 1

    def build_generator(self):
      model = Sequential()
      model.add(Dense(128 * 7 * 7, activation='relu', input_dim=self.latent_dim))
      model.add(Reshape((7, 7, 128)))

      model.add(UpSampling2D())
      model.add(Conv2D(128, kernel_size=3, padding="same"))
      model.add(BatchNormalization(momentum=0.8))
      model.add(Activation('relu'))

      model.add(UpSampling2D())
      model.add(Conv2D(64, kernel_size=3, padding='same'))
      model.add(BatchNormalization(momentum=0.8))
      model.add(Activation('relu'))

      model.add(Conv2D(self.channels, kernel_size=3, pading="same"))
      model.add(Activation('tanh'))
      model.summary()

      noise = Input(shape=(self.latent_dim, ))
      img = model(noise)
      return Model(noise, img)




    def build_discriminator(self):
      model = Sequential()
      model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding='same'))
      model.add(LeakyReLU(alpha=0.2))
      model.add(Dropout(0.25))
      model.add(Conv2D(128, kernel_size=3, strides=2, padding = 'same'))
      model.add(BatchNormalization(momentum=0.8))
      model.add(LeakyReLU(alpha=0.2))
      model.add(Dropout(0.25))

      model.add(Conv2D(128, kernel_size=3, strides=2, padding='same'))
      model.add(BatchNormalization(momentum=0.8))
      model.add(LeakyReLU(alpha=0.2))
      model.add(Dropout(0.25))

      model.add(Conv2D(256, kernel_size=3, strides=1, padding='same'))
      model.add(BatchNormalization(momentum=0.8))
      model.add(LeakyReLU(alpha=0.2))
      model.add(Dropout(0.25))
      model.add(Flatten())
      model.summary()

      img =Input(shape=self.img_shape)
      validity = model(img)

      return Model(img, validity)


In [20]:

# the training function
def train(self, dataset, epochs, batch_size=256, save_interval=50):
  for epoch in tqdm(range(epochs)):

    for image_batch in dataset:
      self.train_step(image_batch, batch_size)


    if epoch % save_interval == 0:
      self.save_imgs(epoch)


 # the train step


def train_step(self, images, BATCH_SIZE):
  noise = tf.random.normal([BATCH_SIZE, self.latent_dim])


  with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
      generated_images = self.generator(noise, training=True)


      real_output = self.discriminator(images, training=True)
      fake_output = self.discriminator(generated_images, training=True)


      gen_loss = generator_loss(fake_output)
      disc_loss = discriminator_loss(real_output, fake_output)


  # this is the gradient for both discriminator and generator
  gradient_of_generator = gen_tape.gradient(gen_loss, self.generator.trainable_variables)
  gradient_of_discriminator = disc_tape.gradient(disc_loss, self.discriminator.trainable_variables)


  # here we apply the gradient
  self.generator_optimizer.apply_gradients(zip(gradient_of_generator, self.generator.trainable_variables))
  self.discriminator_optimizer.apply_gradients(zip(gradient_of_discriminator, self.discriminator.trainable_variables))



In [8]:
# saving the images
def save_imgs(self, epoch):
  r, c = 4, 4
  gen_imgs = self.generator(input_noise, training=False)

  fig = plt.figure(figsize=(4, 4))

  for i in range(gen_imgs.shape[0]):
    plt.subplot(4, 4, i+1)
    plt.imshow(gen_imgs[i, :, :, 0]* 127.5 + 127.5, cmap='gray')
    plt.axis('off')
  fig.savefig("images/dcgan_mnist_%d.png" % epoch)
  plt.close()

In [9]:
(X_train, _), (_, _) = mnist.load_data()

X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype('float32')

Xx_train = (X_train - 127.5) / 127.5

train_dataset = tf.data.Dataset.from_tensor_slices(X_train).shuffle(len(X_train)).batch(batch_size)


11490434/11490434 [==============================] - 0s 0us/step


In [ ]:
dcgan = DCGAN(28, 28, 1)
dcgan.train(train_dataset, epochs=50, batch_size=batch_size, save_interval=5)